In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
# plotting method

def plot_tdf_(df_, tdf_, side_: int = 0, liq_thr=0.1):
    df_.loc[tdf_.index[0]:tdf_.index[-1]].price.plot(style='y-')
    if side_ >= 0:
        longs = tdf_[tdf_.side == 'long']
        le = longs[longs.type == 'entry']
        lc = longs[longs.type == 'close']
        ls = longs[longs.type == 'stop_loss']
        ls.price.plot(style='gx')
        le.price.plot(style='b.')
        longs.pos_price.plot(style='b--')
        if 'close_price' in longs.columns:
            longs.close_price.plot(style='r--')
        lc.price.plot(style='ro')
    if side_ <= 0:
        shrts = tdf_[tdf_.side == 'shrt']
        se = shrts[shrts.type == 'entry']
        sc = shrts[shrts.type == 'close']
        ss = shrts[shrts.type == 'stop_loss']
        ss.price.plot(style='gx')
        se.price.plot(style='r.')
        shrts.pos_price.plot(style='r--')
        if 'close_price' in shrts.columns:
            shrts.close_price.plot(style='b--')
        sc.price.plot(style='bo')
    if 'liq_price' in tdf_.columns:
        tdf_.liq_price.where((tdf_.price - tdf_.liq_price).abs() / tdf_.price < liq_thr, np.nan).plot(style='k--')


In [ ]:
exchange = 'binance'
symbol = 'XMRUSDT'
backtesting_results_dir = f'backtesting_results/{exchange}/{symbol}/'
print('sessions')
sessions = sorted(os.listdir(backtesting_results_dir))
sessions

In [ ]:
# select session, default is last of sorted sessions list
session_name = sessions[-1]
print('session', session_name)
session_dir = f"{backtesting_results_dir}{session_name}/"
results_filepath = f"{session_dir}results.txt"
settings = json.load(open(session_dir + 'backtesting_settings.json'))
with open(results_filepath) as f:
    results = [json.loads(line) for line in f.readlines()]
rdf = pd.DataFrame(results)
print('\nn completed backtests', len(rdf))
rdf.sort_values('gain', ascending=False)

In [ ]:
n_days = settings['n_days']
trades_list_cache_filepath = f"{session_dir}{n_days}_days_trades_list_cache.npy"
print('loading trades list...')
trades_list = np.load(trades_list_cache_filepath, allow_pickle=True)
print('done')
df = pd.DataFrame(list(trades_list))

In [ ]:
live_settings = json.load(open(session_dir + 'best_result_live_settings.json'))
print('live settings, best candidate')
print(json.dumps(live_settings, indent=4))

In [ ]:
df.price.iloc[::100].plot()

In [ ]:
rdfs = rdf.sort_values('gain', ascending=False)
rdfs.head(20)

In [ ]:
best = json.load(open(session_dir + 'best.json'))
ranges = json.load(open(session_dir + 'ranges.json'))
best = {k_: best[k_] for k_ in sorted(ranges)}
# choose backtest to view.
# iloc[0] is first, iloc[1] is second, etc
backtest_result = dict(rdfs.iloc[0])
best = {k: backtest_result[k] for k in best}

tdf = pd.read_csv(f"{session_dir}backtest_trades/{backtest_result['key']}.csv").set_index('trade_id')
for k in backtest_result:
    if k in settings:
        settings[k] = backtest_result[k]
        print(k, settings[k])
settings

In [ ]:
# analyze results
longs = tdf[tdf.side == 'long']
shrts = tdf[tdf.side == 'shrt']
le = longs[longs.type == 'entry']
lc = longs[longs.type == 'close']
se = shrts[shrts.type == 'entry']
sc = shrts[shrts.type == 'close']

biggest_pos_size = tdf.pos_size.abs().max()
pnl_sum = tdf.pnl.sum()
stop_loss_closes = tdf[tdf.type == 'stop_loss']
loss_sum = stop_loss_closes.pnl.sum()
gain = (settings['starting_balance'] + pnl_sum) / settings['starting_balance']
closest_liq = ((tdf.price - tdf.liq_price).abs() / tdf.price).min()
n_stop_loss = len(stop_loss_closes)
n_days = settings['n_days']
average_daily_gain = gain ** (1 / n_days)
closes = tdf[tdf.type == 'close']
print('net pnl', round(pnl_sum, 6))
print('loss sum', round(loss_sum, 6))
print(f'gain {gain * 100 - 100:.2f}%')
print('n_days', n_days)
print(f'average_daily_gain {(average_daily_gain - 1) * 100:.2f}%')
print('n trades', len(tdf))
print('n closes', len(closes))
print('n stop loss closes', n_stop_loss)
print('biggest_pos_size', biggest_pos_size)
print(f'closest liq {closest_liq * 100:.4f}%')


In [ ]:
start_ = 0.0
end_ = 1.0
plot_tdf_(df, tdf.iloc[int(len(tdf) * start_):int(len(tdf) * end_)])

In [ ]:
tdf.pnl.cumsum().plot()

In [ ]:
tdf.pos_size.plot()

In [ ]:
# visualize behavior
step = 120
i = -step

In [ ]:
i += step
tdfc = tdf.iloc[i:i+step]
plot_tdf_(df, tdf.iloc[i:i+step], liq_thr=0.05)

In [ ]:
tdfcj = tdfc.join(pd.Series(tdfc.price.diff(), name='price_diff'))
tdfcj.head(60)

In [ ]:
tdfcj.tail(60)

In [ ]:
closest_liqs = ((tdf.liq_price - tdf.price).abs() / tdf.price).sort_values()
closest_liqs.head()

In [ ]:
i = 0
iloc_ = tdf.index.get_loc(closest_liqs.index[i])
iminus = 20
iplus = 50
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
tdfc.head(60)

In [ ]:
# inspect biggest pos sizes
pos_size_abs = tdf.pos_size.abs().sort_values(ascending=False)
pos_size_abs.head(10)

In [ ]:
i = 0
iloc_ = tdf.index.get_loc(pos_size_abs.index[i])
iminus = 400
iplus = 20
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
ik = -1
stepp = 60

In [ ]:
ik += 1
tdfc.iloc[stepp * ik:stepp * (ik + 1)]

In [ ]:
tdf.tail(30)